In [1]:
from pyspark.sql import SparkSession 
spark=SparkSession.builder.appName('rc').getOrCreate()

22/08/19 23:07:58 WARN Utils: Your hostname, faouzi-Inspiron-15-3567 resolves to a loopback address: 127.0.1.1; using 192.168.1.17 instead (on interface wlp2s0)
22/08/19 23:07:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/19 23:07:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/19 23:08:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql.functions import *

In [3]:
df=spark.read.csv('movie_ratings_df.csv',inferSchema=True,header=True)

In [4]:
print((df.count(),len(df.columns)))

(100000, 3)


In [5]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)



In [6]:
# showing a sample of the dataframe
df.orderBy(rand()).show(10,False)

+------+--------------------------------+------+
|userId|title                           |rating|
+------+--------------------------------+------+
|207   |Three Colors: Red (1994)        |4     |
|90    |Bringing Up Baby (1938)         |5     |
|23    |Aladdin (1992)                  |4     |
|56    |Private Benjamin (1980)         |3     |
|752   |Cop Land (1997)                 |5     |
|580   |Desperate Measures (1998)       |3     |
|271   |Citizen Kane (1941)             |3     |
|48    |Silence of the Lambs, The (1991)|5     |
|773   |Tie Me Up! Tie Me Down! (1990)  |3     |
|265   |Rock, The (1996)                |5     |
+------+--------------------------------+------+
only showing top 10 rows



In [7]:
#check number of ratings by each user
df.groupBy('userId').count().orderBy('count',ascending=False).show(11,False)

+------+-----+
|userId|count|
+------+-----+
|405   |737  |
|655   |685  |
|13    |636  |
|450   |540  |
|276   |518  |
|416   |493  |
|537   |490  |
|303   |484  |
|234   |480  |
|393   |448  |
|181   |435  |
+------+-----+
only showing top 11 rows



In [8]:
#number of times movie been rated 
df.groupBy('title').count().orderBy('count',ascending=False).show(10,False)

+-----------------------------+-----+
|title                        |count|
+-----------------------------+-----+
|Star Wars (1977)             |583  |
|Contact (1997)               |509  |
|Fargo (1996)                 |508  |
|Return of the Jedi (1983)    |507  |
|Liar Liar (1997)             |485  |
|English Patient, The (1996)  |481  |
|Scream (1996)                |478  |
|Toy Story (1995)             |452  |
|Air Force One (1997)         |431  |
|Independence Day (ID4) (1996)|429  |
+-----------------------------+-----+
only showing top 10 rows



In [9]:
from pyspark.ml.feature import StringIndexer,IndexToString

In [10]:
#convert the movie title columns into numerical values
stringIndexer = StringIndexer(inputCol="title", outputCol="title_new")

In [11]:
model = stringIndexer.fit(df)

In [12]:
#creating a new dataframe the numerical values
indexed = model.transform(df)

In [13]:
#validate the numerical title values
indexed.show(10)

+------+------------+------+---------+
|userId|       title|rating|title_new|
+------+------------+------+---------+
|   196|Kolya (1996)|     3|    287.0|
|    63|Kolya (1996)|     3|    287.0|
|   226|Kolya (1996)|     5|    287.0|
|   154|Kolya (1996)|     3|    287.0|
|   306|Kolya (1996)|     5|    287.0|
|   296|Kolya (1996)|     4|    287.0|
|    34|Kolya (1996)|     5|    287.0|
|   271|Kolya (1996)|     4|    287.0|
|   201|Kolya (1996)|     4|    287.0|
|   209|Kolya (1996)|     4|    287.0|
+------+------------+------+---------+
only showing top 10 rows



In [14]:
#split the data into training and test datatset with 75% , 25% ratio
train,test=indexed.randomSplit([0.75,0.25])

In [15]:
# number of rows in the training set
train.count()

74854

In [16]:
#number of rows in the testing set
test.count()

25146

In [17]:
from pyspark.ml.recommendation import ALS

In [18]:
#Training a new recommender model using the training datatset
rec=ALS(maxIter=10,regParam=0.01,userCol='userId',itemCol='title_new',ratingCol='rating',nonnegative=True,coldStartStrategy="drop")

In [19]:
#fit the model on the training set
rec_model=rec.fit(train)

In [20]:
#making predictions on test set 
predicted_ratings=rec_model.transform(test)

In [21]:
#columns in predicted ratings dataframe
predicted_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- title_new: double (nullable = false)
 |-- prediction: float (nullable = false)



In [22]:
#predicted vs actual ratings for test set 
predicted_ratings.orderBy(rand()).show(10)

+------+--------------------+------+---------+----------+
|userId|               title|rating|title_new|prediction|
+------+--------------------+------+---------+----------+
|    13|Conan the Barbari...|     4|    311.0|  2.639549|
|   363|Cook the Thief Hi...|     2|    402.0| 4.0316243|
|   433|Three Colors: Red...|     5|    401.0| 4.6939774|
|    49|        Evita (1996)|     4|     60.0| 1.5356435|
|   388|Vegas Vacation (1...|     2|    437.0|  4.614728|
|   342|      Aladdin (1992)|     4|     95.0|  3.367646|
|   386|    Boot, Das (1981)|     5|    116.0| 4.8781157|
|   224|Conspiracy Theory...|     4|     39.0|  3.240925|
|   768|    Rock, The (1996)|     4|     17.0| 3.3611038|
|   342|Trainspotting (1996)|     5|     68.0| 3.9960408|
+------+--------------------+------+---------+----------+
only showing top 10 rows



In [23]:
#importing Regression Evaluator to measure RMSE
from pyspark.ml.evaluation import RegressionEvaluator

In [24]:
#create Regressor evaluator object for measuring accuracy
evaluator=RegressionEvaluator(metricName='rmse',predictionCol='prediction',labelCol='rating')

In [25]:
#apply the RE on predictions dataframe to calculate RMSE
rmse=evaluator.evaluate(predicted_ratings)

In [26]:
# RMSE error is higher in case of smaller datasets
print(rmse)

1.0212116210449953


In [27]:
# Recommending movies to users

In [28]:
#create dataset of all distinct movies 
unique_movies=indexed.select('title_new').distinct()
unique_movies.count()

1664

In [29]:
a = unique_movies.alias('a')

In [30]:
user_id=85

In [31]:
#movies already watched and rated 
watched_movies=indexed.filter(indexed['userId'] == user_id).select('title_new').distinct()
watched_movies.count()

287

In [32]:
b=watched_movies.alias('b')

In [33]:
#joining both tables on a left join 
total_movies = a.join(b, a.title_new == b.title_new,how='left')
total_movies.show(10,False)

+---------+---------+
|title_new|title_new|
+---------+---------+
|305.0    |305.0    |
|596.0    |null     |
|299.0    |null     |
|769.0    |null     |
|692.0    |null     |
|934.0    |null     |
|1051.0   |null     |
|496.0    |null     |
|558.0    |558.0    |
|170.0    |null     |
+---------+---------+
only showing top 10 rows



In [34]:
# movies not yet rated
remaining_movies=total_movies.where(col("b.title_new").isNull()).select(a.title_new).distinct()
remaining_movies.count()

1377

In [35]:
#adding new column of user_Id of active useer to remaining movies df 
remaining_movies=remaining_movies.withColumn("userId",lit(int(user_id)))
remaining_movies.show(10,False)

+---------+------+
|title_new|userId|
+---------+------+
|596.0    |85    |
|299.0    |85    |
|769.0    |85    |
|692.0    |85    |
|934.0    |85    |
|1051.0   |85    |
|496.0    |85    |
|170.0    |85    |
|184.0    |85    |
|576.0    |85    |
+---------+------+
only showing top 10 rows



In [36]:
#making recommendations using ALS recommender model and selecting only top 'n' movies
recommendations=rec_model.transform(remaining_movies).orderBy('prediction',ascending=False)
recommendations.show(5,False)

+---------+------+----------+
|title_new|userId|prediction|
+---------+------+----------+
|1265.0   |85    |5.343667  |
|944.0    |85    |4.849672  |
|1311.0   |85    |4.8482637 |
|1288.0   |85    |4.7528167 |
|1347.0   |85    |4.727878  |
+---------+------+----------+
only showing top 5 rows



In [37]:
#restitution of string names for movie titles
movie_title = IndexToString(inputCol="title_new", outputCol="title",labels=model.labels)
final_recommendations=movie_title.transform(recommendations)
final_recommendations.show(10,False)

+---------+------+----------+---------------------------------------------------------------------------------+
|title_new|userId|prediction|title                                                                            |
+---------+------+----------+---------------------------------------------------------------------------------+
|1265.0   |85    |5.343667  |Duoluo tianshi (1995)                                                            |
|944.0    |85    |4.849672  |Steel (1997)                                                                     |
|1311.0   |85    |4.8482637 |Hearts and Minds (1996)                                                          |
|1288.0   |85    |4.7528167 |Whole Wide World, The (1996)                                                     |
|1347.0   |85    |4.727878  |Angel Baby (1995)                                                                |
|1322.0   |85    |4.6773515 |Old Lady Who Walked in the Sea, The (Vieille qui marchait dans la mer, La) 

In [38]:
# function to recommend top 'n' movies to any particular user from the dataframe
def Recommend(user_id,n):
    
    a = unique_movies.alias('a')
    
    watched_movies=indexed.filter(indexed['userId'] == user_id).select('title_new')
    
    b=watched_movies.alias('b')
    
    total_movies = a.join(b, a.title_new == b.title_new,how='left')
    
    remaining_movies=total_movies.where(col("b.title_new").isNull()).select(a.title_new).distinct()
    
    remaining_movies=remaining_movies.withColumn("userId",lit(int(user_id)))
    
    recommendations=rec_model.transform(remaining_movies).orderBy('prediction',ascending=False).limit(n)    
    
    movie_title = IndexToString(inputCol="title_new", outputCol="title",labels=model.labels)
    final_recommendations=movie_title.transform(recommendations)
    
    return final_recommendations.show(n,False)

In [39]:
Recommend(55,15)

+---------+------+----------+---------------------------------------------------------------------------------+
|title_new|userId|prediction|title                                                                            |
+---------+------+----------+---------------------------------------------------------------------------------+
|1322.0   |55    |15.781107 |Old Lady Who Walked in the Sea, The (Vieille qui marchait dans la mer, La) (1991)|
|865.0    |55    |13.048766 |Burnt By the Sun (1994)                                                          |
|980.0    |55    |12.547605 |Shall We Dance? (1937)                                                           |
|679.0    |55    |12.226465 |Flirting With Disaster (1996)                                                    |
|1283.0   |55    |12.18191  |Secret Agent, The (1996)                                                         |
|944.0    |55    |12.116684 |Steel (1997)                                                               